In [2]:
from decimal import Decimal, getcontext, ROUND_HALF_UP, ROUND_HALF_EVEN, ROUND_FLOOR
import ipywidgets as widgets
from IPython.display import display, clear_output
import re

getcontext().prec = 40
getcontext().rounding = ROUND_HALF_UP

INTERMEDIATE_MIN = Decimal('-10000000000000.0000000000')
INTERMEDIATE_MAX = Decimal('10000000000000.0000000000')
FINAL_MIN = Decimal('-10000000000000.000000')
FINAL_MAX = Decimal('10000000000000.000000')

def parse_number(input_str: str) -> Decimal:
    s = input_str.strip()
    if not s:
        raise ValueError("Пустой ввод")
    
    no_spaces = s.replace(' ', '')
    normalized = no_spaces.replace(',', '.')
    
    try:
        Decimal(normalized)
    except:
        raise ValueError("Некорректное число (буквы, лишние символы)")
    
    if re.search(r'[+-].*[+-]', no_spaces):
        raise ValueError("Некорректный знак (например, '0.0-1')")
    
    if '.' in no_spaces and ',' in no_spaces:
        raise ValueError("Одновременно точка и запятая")
    
    frac_sep = '.' if '.' in s else ',' if ',' in s else None
    original_int = s.split(frac_sep)[0].strip() if frac_sep else s.strip()
    
    if original_int.startswith(('-', '+')):
        original_int = original_int[1:].lstrip()
    
    if ' ' in original_int:
        groups = [g for g in original_int.split(' ') if g]
        if not groups:
            raise ValueError("Неверные пробелы")
        for group in groups[1:]:
            if len(group) != 3 or not group.isdigit():
                raise ValueError("Пробелы только для групп по 3 цифры")
        if not (1 <= len(groups[0]) <= 3) or not groups[0].isdigit():
            raise ValueError("Левая группа: 1–3 цифры")
    
    return Decimal(normalized)

def format_integer(int_str: str) -> str:
    if not int_str or int_str in ['-', '+']:
        return '0'
    sign = '-' if int_str.startswith('-') else ''
    int_str = int_str.lstrip('-+')
    
    formatted = ''
    for i, d in enumerate(reversed(int_str)):
        if i > 0 and i % 3 == 0:
            formatted = ' ' + formatted
        formatted = d + formatted
    return sign + (formatted or '0')

def format_result(value: Decimal) -> str:
    rounded = value.quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP)
    s = f"{rounded:f}"
    
    if '.' in s:
        int_part, frac_part = s.split('.')
        frac_clean = frac_part.rstrip('0')
        if frac_clean:
            return format_integer(int_part) + '.' + frac_clean
        else:
            return format_integer(int_part)
    return format_integer(s)


title = widgets.HTML("<h2 style='text-align:center; color:#00bfff; margin:30px 0 20px 0;'>Финансовый калькулятор</h2>")
info = widgets.HTML("<div style='text-align:center; margin-bottom:30px; color:#ccc;'>"
                    "Крепская Анна Валерьевна | Курс: 4 | Группа: 4 | Год: 2025"
                    "</div>")

num1 = widgets.Text(value='0', description='Число 1:', layout=widgets.Layout(width='180px'))
op1 = widgets.Dropdown(options=['+', '–', '×', '÷'], value='+', layout=widgets.Layout(width='60px'))

left_bracket = widgets.HTML("<span style='font-size:28px; font-weight:bold; margin:0 10px;'> ( </span>")
num2 = widgets.Text(value='0', description='Число 2:', layout=widgets.Layout(width='180px'))
op2 = widgets.Dropdown(options=['+', '–', '×', '÷'], value='+', layout=widgets.Layout(width='60px'))
num3 = widgets.Text(value='0', description='Число 3:', layout=widgets.Layout(width='180px'))
right_bracket = widgets.HTML("<span style='font-size:28px; font-weight:bold; margin:0 10px;'> ) </span>")

op3 = widgets.Dropdown(options=['+', '–', '×', '÷'], value='+', layout=widgets.Layout(width='60px'))
num4 = widgets.Text(value='0', description='Число 4:', layout=widgets.Layout(width='180px'))

expression = widgets.HBox([
    num1, op1, left_bracket, num2, op2, num3, right_bracket, op3, num4
], layout=widgets.Layout(justify_content='center', margin='20px 0'))

rounding_type = widgets.RadioButtons(
    options=[
        ('Математическое', ROUND_HALF_UP),
        ('Бухгалтерское (банковское)', ROUND_HALF_EVEN),
        ('Усечение', ROUND_FLOOR)
    ],
    value=ROUND_HALF_UP,
    layout=widgets.Layout(width='400px')
)

rounding_box = widgets.HBox([
    widgets.Label("Округление до целых:", layout=widgets.Layout(width='200px')),
    rounding_type
], layout=widgets.Layout(justify_content='center', margin='30px 0'))

button = widgets.Button(description='Вычислить', button_style='primary',
                        layout=widgets.Layout(width='250px', height='50px'))

output = widgets.Output(layout=widgets.Layout(
    border='1px solid #555',
    padding='20px',
    margin='30px auto',
    width='700px',
    min_height='120px',
    background_color='#111',
    font_family='monospace',
    font_size='16px'
))

def calculate(_):
    with output:
        clear_output()
        try:
            a = parse_number(num1.value)
            b = parse_number(num2.value)
            c = parse_number(num3.value)
            d = parse_number(num4.value)
            
            ops = {'+': lambda x, y: x + y,
                   '–': lambda x, y: x - y,
                   '×': lambda x, y: x * y,
                   '÷': lambda x, y: x / y if y != 0 else raise_div_zero()}

            if op2.value == '÷' and c == 0:
                raise ValueError("Деление на ноль в скобках")
            inner = ops[op2.value](b, c)
            inner_rounded = inner.quantize(Decimal('1e-10'), rounding=ROUND_HALF_UP)
            if not (INTERMEDIATE_MIN <= inner_rounded <= INTERMEDIATE_MAX):
                raise ValueError("Переполнение в скобках")
            
            terms = [a, inner_rounded, d]
            operators = [op1.value, op3.value]
            
            i = 0
            while i < len(operators):
                if operators[i] in ['×', '÷']:
                    if operators[i] == '÷' and terms[i+1] == 0:
                        raise ValueError(f"Деление на ноль в операции {operators[i]}")
                    result = ops[operators[i]](terms[i], terms[i+1])
                    result_rounded = result.quantize(Decimal('1e-10'), rounding=ROUND_HALF_UP)
                    if not (INTERMEDIATE_MIN <= result_rounded <= INTERMEDIATE_MAX):
                        raise ValueError(f"Переполнение в операции {operators[i]}")
 
                    terms[i] = result_rounded
                    del terms[i+1]
                    del operators[i]
                else:
                    i += 1
            
            result = terms[0]
            for i in range(len(operators)):
                if operators[i] == '÷' and terms[i+1] == 0:  
                    raise ValueError("Деление на ноль")
                result = ops[operators[i]](result, terms[i+1])
                result = result.quantize(Decimal('1e-10'), rounding=ROUND_HALF_UP)
                if not (INTERMEDIATE_MIN <= result <= INTERMEDIATE_MAX):
                    raise ValueError(f"Переполнение в операции {operators[i]}")
            
            final = result
            
            if not (FINAL_MIN <= final <= FINAL_MAX):
                raise ValueError("Итоговый результат вне диапазона")
            
            formatted = format_result(final)
            print(f"Результат вычисления: {formatted}\n")
            
            getcontext().rounding = rounding_type.value
            rounded_int = final.to_integral_value()
            method = [x[0] for x in rounding_type.options if x[1] == rounding_type.value][0]
            print(f"Округлённый до целых ({method}): {format_integer(str(rounded_int))}")
            
        except ValueError as e:
            print(f"Ошибка: {str(e)}")
        except Exception as e:
            print(f"Неизвестная ошибка: {str(e)}")

def raise_div_zero():
    raise ValueError("Деление на ноль")

button.on_click(calculate)

display(widgets.VBox([
    title,
    info,
    expression,
    rounding_box,
    widgets.HBox([button], layout=widgets.Layout(justify_content='center', margin='20px 0')),
    output
], layout=widgets.Layout(align_items='center')))